In [1]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
#import keras as keras

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
XYDataFileName = "42p_82894samples_06-07-24_AHDincidence_XY.txt";
dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data";
mypara = {'num_epochs': 50, 'batch_size': 20}
'''
Run MGM
Note: MGM was implemented in Java and the following Python APIs call the Java implementation.
Please restart the Python program after encountering a JVM problem.
The format of the input data file must be ".txt" in which columns are separated by "\t" and it should also include the response variables.
Here is what the input data should look like:
X1 X2 ... Xp
1  1  ... 1
'''
# import the MGM package
#from MGM.MGM import MGM
# Initialize a MGM object
#mgm = MGM();
'''
Run MGM
Parameters:
    dataFolderPath: the directory that stores the input data.
    DataFileName: the name of the input data.
    lambda_continuous_continuous: the panalty value 'lambda' set for the associations whose two end variables are continuous.
    lamda_continuous_discrete: the panalty value 'lambda' set for the associations whose one end variable is continuous and the other is discrete.
    lamda_discrete_discrete: the panalty value 'lambda' set for the associations whose two end variables are discrete.
    
Return:
    mgmOutputFile: a tuple, where the first file contains all the selected associations and the second file contains the corresponding likelihoods.
'''
#mgmOutputFile = mgm.runMGM(dataFolderPath, XYDataFileName,lambda_continuous_continuous = 0.3, lamda_continuous_discrete = 0.3, lamda_discrete_discrete = 0.3);
"""
MGM uses the Python package Jpype to call MGM's Java implementation.
According to Jpype documents, it says "Due to limitations in the JPype, 
it is not possible to restart the JVM after being terminated."
Therefore, please restart the Python kernel if you encounter an OSError (i.e., "OSError: JVM cannot be restarted").
"""
#print("MGM's output was saved as the following file:");
#print(mgmOutputFile[0]);
#print("The likelihood values were saved as the following file:");
#print(mgmOutputFile[1]);

'\nMGM uses the Python package Jpype to call MGM\'s Java implementation.\nAccording to Jpype documents, it says "Due to limitations in the JPype, \nit is not possible to restart the JVM after being terminated."\nTherefore, please restart the Python kernel if you encounter an OSError (i.e., "OSError: JVM cannot be restarted").\n'

In [3]:
'''
#Generate knockoff data using one of two methods: ISEE Omega and Cholesky_LU.
The code for generating ISEE Omega knockoff is implemented using R. Please make sure your computer has R installed.
'''
from DL.knockoff.KnockoffGenerator import KnockoffGenerator;
generator = KnockoffGenerator();

dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data";
DataFileName = "42p_82894samples_06-07-24_AHDincidence_X.txt"
knockoffFilePath = generator.CholLuKnockoff(dataFolderPath, DataFileName,sep="\t");

# #If want to generate ISEE Omega knockoff, please set the ISEE code path and R home environment.
# generator.set_ISEE_path("/absolute_path_of_DAG_DeepVASE");#/home/user/DAG_DeepVASE/
# generator.set_R_home('absolute_path_to_directory_where_r_is_installed');#e.g.,/home/user/lib/R
#knockoffFilePath = generator.ISEEKnockoff(dataFolderPath, DataFileName,sep="\t");

print("The newly generated knockoff file is named as:");
print(knockoffFilePath);

KnockoffGenerator__init__
(82894, 42)
['K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'K8', 'K9', 'K10', 'K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20', 'K21', 'K22', 'K23', 'K24', 'K25', 'K26', 'K27', 'K28', 'K29', 'K30', 'K31', 'K32', 'K33', 'K34', 'K35', 'K36', 'K37', 'K38', 'K39', 'K40', 'K41', 'K42']
The newly generated knockoff file is named as:
/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/42p_82894samples_06-07-24_AHDincidence_X_chol_lu_knockoff.txt


In [4]:
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t"); #for stat knockoffs
#XKnockoffData = pd.read_csv("/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/06-07-24-onlyIncidenceAHD_PRSknockoffs.csv", sep=",") #for PRS knockoffs
#XKnockoffData

In [5]:
''''''
# After generating the knockoff data, run DNN

dataFolderPath = "/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data"
XKnockoffData = pd.read_csv(knockoffFilePath,sep="\t"); #for stat knockoffs
#XKnockoffData = pd.read_csv("/Users/adsriram98/Documents/PARK_LAB/PRS_calc/data/06-07-24-onlyIncidenceAHD_PRSknockoffs.csv", sep=",") #for PRS knockoffs

YDataFileName = "42p_82894samples_06-07-24_AHDincidence_Y.txt";
Ydata = pd.read_csv(dataFolderPath+os.path.sep+YDataFileName,sep="\t");

XKValues = XKnockoffData.values;
YValues = Ydata.values;
    
pNum = int(XKValues.shape[1] / 2);
n = XKValues.shape[0];
print(XKValues.shape);
print(YValues.shape);
print(pNum);
    
XOrigin = XKValues[:, 0:pNum];
knockoff = XKValues[:, pNum:];

X3DTrain = np.zeros((n, pNum, 2));
X3DTrain[:, :, 0] = XOrigin;
X3DTrain[:, :, 1] = knockoff;
labelTrain = YValues;
    
coeff = 0.05 * np.sqrt(2.0 * np.log(pNum) / n);

nOutputs = Ydata.shape[1];

#Save the DNN output to the following directory.
resultDir = dataFolderPath+os.path.sep+'08_18_predictor_STATKO_DNN_result/';
if not os.path.exists(resultDir):
    os.makedirs(resultDir);
    
from DL.DNN.DNN import DNN;
#from sklearn.model_selection import cross_val_score;
#import tensorflow as tf;

paralabel = "epo" + str(mypara['num_epochs']) + "batch" + str(mypara['batch_size']);
print(paralabel);

dnn = DNN(output_layer_activation='sigmoid', output_loss = 'binary_crossentropy',
          num_epochs = mypara['num_epochs'],
          batch_size = mypara['batch_size']);

(82894, 84)
(82894, 1)
42
epo50batch20
__init__parameters


In [6]:
X3DTrain.shape

(82894, 42, 2)

In [7]:
X3DTrain.shape
labelTrain.shape

(82894, 1)

In [8]:
#labelTrain
XKValues

array([[ 1.2686    ,  0.941     , 24.9653    , ...,  0.35780011,
        -0.47219762,  0.3752051 ],
       [ 1.3666    ,  0.994     , 32.5852    , ...,  0.68682114,
        -1.05508579,  1.04861109],
       [ 1.0478    ,  1.234     , 31.2816    , ..., -0.12149252,
         1.22356894,  0.08896952],
       ...,
       [ 1.3414    ,  0.775     , 30.2663    , ...,  0.31485929,
         0.49313656,  0.74606313],
       [ 1.1477    ,  0.993     , 28.68      , ..., -0.92192622,
         0.98724057, -0.16921804],
       [ 1.5599    ,  0.758     , 23.0546    , ..., -0.06214427,
         0.16534002,  0.74170058]])

In [9]:
#import tensorflow as tf;
#scores= cross_val_score(model, X3DTrain, labelTrain, cv=5)
Ydata.shape

(82894, 1)

In [10]:
if not os.path.exists(dataFolderPath +os.path.sep+paralabel):
        os.makedirs(dataFolderPath +os.path.sep+paralabel)
        print(f"Directory '{dataFolderPath +os.path.sep+paralabel}' created successfully.")

In [11]:

#yhat3DTrain=model.predict(X3DTrain)
#pd.DataFrame(yhat3DTrain).to_csv(dataFolderPath +os.path.sep+paralabel+os.path.sep+"test.csv")

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, matthews_corrcoef, precision_recall_curve, average_precision_score
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbalanced_pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labelTrain), y=labelTrain.ravel())
class_weights = dict(enumerate(class_weights))


def load_data():
    data = X3DTrain
    labels = labelTrain
    header_info = "Your header information here"
    return data, labels, header_info

def create_model():
    dnn = DNN(output_layer_activation='sigmoid', output_loss='binary_crossentropy',
              num_epochs=mypara['num_epochs'], batch_size=mypara['batch_size'])
    model = dnn.build_DNN(pNum, nOutputs, coeff)
    callback = DNN.Job_finish_Callback(resultDir, pNum)
    return model, callback

def calculate_optimal_threshold(y_true, y_probs):
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

def train_and_evaluate_xgboost(data_train, labels_train, data_test, labels_test, metrics_vector_xgb):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    dtrain = xgb.DMatrix(data_train_flattened, label=labels_train)
    dtest = xgb.DMatrix(data_test_flattened, label=labels_test)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'scale_pos_weight': class_weights[1] / class_weights[0]
    }
    num_rounds = 100

    bst = xgb.train(params, dtrain, num_rounds)
    preds_prob = bst.predict(dtest)
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_xgb.append([auc_score, pr_auc, mcc])

def train_and_evaluate_rf(data_train, labels_train, data_test, labels_test, metrics_vector_rf):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    rf_model = RandomForestClassifier(class_weight='balanced', n_estimators=100)
    rf_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = rf_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_rf.append([auc_score, pr_auc, mcc])

def train_and_evaluate_lr(data_train, labels_train, data_test, labels_test, metrics_vector_lr):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    lr_model = LogisticRegression(class_weight='balanced', max_iter=5000)
    lr_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = lr_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_lr.append([auc_score, pr_auc, mcc])
    
def train_and_evaluate_svc(data_train, labels_train, data_test, labels_test, metrics_vector_svc):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    svc_model = LinearSVC(class_weight='balanced', max_iter=5000)
    calibrated_svc = CalibratedClassifierCV(svc_model, method='sigmoid', cv=5)
    calibrated_svc.fit(data_train_flattened, labels_train.ravel())
    preds_prob = calibrated_svc.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_svc.append([auc_score, pr_auc, mcc])

def train_and_evaluate_knn(data_train, labels_train, data_test, labels_test, metrics_vector_knn):
    data_train_flattened = data_train.reshape(data_train.shape[0], -1)
    data_test_flattened = data_test.reshape(data_test.shape[0], -1)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(data_train_flattened, labels_train.ravel())
    preds_prob = knn_model.predict_proba(data_test_flattened)[:, 1]
    auc_score = roc_auc_score(labels_test, preds_prob)
    precision, recall, _ = precision_recall_curve(labels_test, preds_prob)
    pr_auc = auc(recall, precision)
    optimal_threshold = calculate_optimal_threshold(labels_test, preds_prob)
    mcc = matthews_corrcoef(labels_test, (preds_prob > optimal_threshold).astype(int))

    metrics_vector_knn.append([auc_score, pr_auc, mcc])

def train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, metrics_vector):
    smote = SMOTE(sampling_strategy='auto')
    under_sampler = RandomUnderSampler(sampling_strategy='auto')
    
    resampling_pipeline = imbalanced_pipeline([
        ('smote', smote),
        ('under', under_sampler)
    ])
    
    data_train_resampled, labels_train_resampled = resampling_pipeline.fit_resample(data_train.reshape(data_train.shape[0], -1), labels_train.ravel())
    data_train_resampled = data_train_resampled.reshape(data_train_resampled.shape[0], pNum, 2)
    
    dnn.train_DNN(model, data_train_resampled, labels_train_resampled, callback, class_weights=class_weights)
    yhat_test_prob = model.predict(data_test)

    fpr, tpr, thresholds = roc_curve(labels_test, yhat_test_prob)
    roc_auc = auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    yhat_test = (yhat_test_prob >= optimal_threshold).astype(int)

    true_values = pd.Series(labels_test.ravel())
    predicted_values = pd.Series(yhat_test.ravel())
    label_encoder = LabelEncoder()
    true_values = label_encoder.fit_transform(true_values)
    predicted_values = label_encoder.transform(predicted_values)

    cm = confusion_matrix(true_values, predicted_values)
    tn, fp, fn, tp = cm.ravel()
    accuracy = accuracy_score(true_values, predicted_values)
    sensitivity = recall_score(true_values, predicted_values)
    specificity = tn / (tn + fp)
    precision = precision_score(true_values, predicted_values)
    f1 = f1_score(true_values, predicted_values)
    mcc = matthews_corrcoef(true_values, predicted_values)
    precision_recall_auc = average_precision_score(true_values, yhat_test_prob)
    
    metrics = [accuracy, sensitivity, specificity, precision, f1, roc_auc, precision_recall_auc, mcc, i]
    metrics_vector.append(metrics)

if __name__ == "__main__":
    n_folds = 10
    data, labels, header_info = load_data()
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True)
    metrics_vector = []
    metrics_vector_xgb = []
    metrics_vector_rf = []
    metrics_vector_lr = []
    metrics_vector_svc = []
    metrics_vector_knn = []

    for i, (train_idx, test_idx) in enumerate(skf.split(data, labels)):
        print("Running Fold", i+1, "/", n_folds)
        data_train, labels_train = data[train_idx], labels[train_idx]
        data_test, labels_test = data[test_idx], labels[test_idx]

        model, callback = create_model()
        train_and_evaluate_model(model, data_train, labels_train, data_test, labels_test, metrics_vector)
        train_and_evaluate_xgboost(data_train, labels_train, data_test, labels_test, metrics_vector_xgb)
        train_and_evaluate_rf(data_train, labels_train, data_test, labels_test, metrics_vector_rf)
        train_and_evaluate_lr(data_train, labels_train, data_test, labels_test, metrics_vector_lr)
        train_and_evaluate_svc(data_train, labels_train, data_test, labels_test, metrics_vector_svc)
        train_and_evaluate_knn(data_train, labels_train, data_test, labels_test, metrics_vector_knn)

    mean_auc_xgb = np.mean([x[0] for x in metrics_vector_xgb])
    mean_pr_auc_xgb = np.mean([x[1] for x in metrics_vector_xgb])
    mean_mcc_xgb = np.mean([x[2] for x in metrics_vector_xgb])

    mean_auc_rf = np.mean([x[0] for x in metrics_vector_rf])
    mean_pr_auc_rf = np.mean([x[1] for x in metrics_vector_rf])
    mean_mcc_rf = np.mean([x[2] for x in metrics_vector_rf])

    mean_auc_lr = np.mean([x[0] for x in metrics_vector_lr])
    mean_pr_auc_lr = np.mean([x[1] for x in metrics_vector_lr])
    mean_mcc_lr = np.mean([x[2] for x in metrics_vector_lr])

    mean_auc_svc = np.mean([x[0] for x in metrics_vector_svc])
    mean_pr_auc_svc = np.mean([x[1] for x in metrics_vector_svc])
    mean_mcc_svc = np.mean([x[2] for x in metrics_vector_svc])

    mean_auc_knn = np.mean([x[0] for x in metrics_vector_knn])
    mean_pr_auc_knn = np.mean([x[1] for x in metrics_vector_knn])
    mean_mcc_knn = np.mean([x[2] for x in metrics_vector_knn])

    print(f"Mean AUC XGBoost: {mean_auc_xgb}")
    print(f"Mean prAUC XGBoost: {mean_pr_auc_xgb}")
    print(f"Mean MCC XGBoost: {mean_mcc_xgb}")

    print(f"Mean AUC Random Forest: {mean_auc_rf}")
    print(f"Mean prAUC Random Forest: {mean_pr_auc_rf}")
    print(f"Mean MCC Random Forest: {mean_mcc_rf}")

    print(f"Mean AUC Logistic Regression: {mean_auc_lr}")
    print(f"Mean prAUC Logistic Regression: {mean_pr_auc_lr}")
    print(f"Mean MCC Logistic Regression: {mean_mcc_lr}")

    print(f"Mean AUC LinearSVC: {mean_auc_svc}")
    print(f"Mean prAUC LinearSVC: {mean_pr_auc_svc}")
    print(f"Mean MCC LinearSVC: {mean_mcc_svc}")

    print(f"Mean AUC KNeighborsClassifier: {mean_auc_knn}")
    print(f"Mean prAUC KNeighborsClassifier: {mean_pr_auc_knn}")
    print(f"Mean MCC KNeighborsClassifier: {mean_mcc_knn}")

    column_names = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'ROC_AUC', 'prAUC', 'MCC', 'Fold']
    df = pd.DataFrame(metrics_vector, columns=column_names)

    for i in range(len(metrics_vector)):
        metrics_vector[i].extend(metrics_vector_xgb[i])
        metrics_vector[i].extend(metrics_vector_rf[i])
        metrics_vector[i].extend(metrics_vector_lr[i])
        metrics_vector[i].extend(metrics_vector_svc[i])
        metrics_vector[i].extend(metrics_vector_knn[i])

    column_names.extend(['XGBoost_AUC', 'XGBoost_prAUC', 'XGBoost_MCC', 'RandomForest_AUC', 'RandomForest_prAUC', 'RandomForest_MCC', 'LogisticRegression_AUC', 'LogisticRegression_prAUC', 'LogisticRegression_MCC', 'LinearSVC_AUC', 'LinearSVC_prAUC', 'LinearSVC_MCC', 'KNeighbors_AUC', 'KNeighbors_prAUC', 'KNeighbors_MCC'])
    df = pd.DataFrame(metrics_vector, columns=column_names)

    mean_row = df.mean(axis=0)
    mean_row['Fold'] = 'Mean'
    df = pd.concat([df, pd.DataFrame([mean_row])], ignore_index=True)

    df.to_csv(dataFolderPath + os.path.sep + paralabel + os.path.sep + "08_18_CHDincidence_STATknockoffs_metrics_summary_dropout005_withXGB_RF_LR_SVC_KNN.csv")


Running Fold 1 / 10
__init__parameters
[layer]: Input	[shape]: [None, 42, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: Flatten	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/50
6758/6767 [============================>.] - ETA: 0s - loss: 0.2408on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 11s 1ms/step - loss: 0.2408
Epoch 2/50
6731/6767 [============================>.] - ETA: 0s - loss: 0.1888on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 10s 1ms/step - loss: 0.188

/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running Fold 2 / 10
__init__parameters
[layer]: Input	[shape]: [None, 42, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: Flatten	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/50
6755/6767 [============================>.] - ETA: 0s - loss: 0.2532on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 11s 1ms/step - loss: 0.2531
Epoch 2/50
6743/6767 [============================>.] - ETA: 0s - loss: 0.1919on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 9s 1ms/step - loss: 0.1919

/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running Fold 3 / 10
__init__parameters
[layer]: Input	[shape]: [None, 42, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: Flatten	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/50
6757/6767 [============================>.] - ETA: 0s - loss: 0.2455on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 11s 1ms/step - loss: 0.2454
Epoch 2/50
6740/6767 [============================>.] - ETA: 0s - loss: 0.2011on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 9s 1ms/step - loss: 0.2010

...

Running Fold 9 / 10
__init__parameters
[layer]: Input	[shape]: [None, 42, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: Flatten	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/50
6732/6767 [============================>.] - ETA: 0s - loss: 0.2177on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 11s 1ms/step - loss: 0.2175
Epoch 2/50
6734/6767 [============================>.] - ETA: 0s - loss: 0.1847on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 9s 1ms/step - loss: 0.1846

/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running Fold 10 / 10
__init__parameters
[layer]: Input	[shape]: [None, 42, 2] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: LocallyConnected1D	[shape]: [None, 42, 1] 

[layer]: Flatten	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 42] 

[layer]: Dense	[shape]: [None, 1] 

Epoch 1/50
6739/6767 [============================>.] - ETA: 0s - loss: 0.2195on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 11s 1ms/step - loss: 0.2194
Epoch 2/50
6743/6767 [============================>.] - ETA: 0s - loss: 0.1895on_epoch_end
h_local1_weight = (42, 2, 1)
h_local2_weight = (42, 1, 1)
h0 = (42, 2)
h0_abs = (42, 2)
h1 = (42, 42)
h2 = (42, 42)
h3 = (42, 1)
W1 = (42, 42)
W2 = (42, 42)
W3 = (42, 1)
6767/6767 [==============================] - 9s 1ms/step - loss: 0.189

/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/adsriram98/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:1244: Conv

Mean AUC XGBoost: 0.7683582633742457
Mean prAUC XGBoost: 0.25335182303563153
Mean MCC XGBoost: 0.24723738430303718
Mean AUC Random Forest: 0.7803576899567803
Mean prAUC Random Forest: 0.2601983022365215
Mean MCC Random Forest: 0.2720792786241966
Mean AUC Logistic Regression: 0.8026228298097621
Mean prAUC Logistic Regression: 0.28520178101087806
Mean MCC Logistic Regression: 0.29095955855031325
Mean AUC LinearSVC: 0.7904188459418704
Mean prAUC LinearSVC: 0.2695914957857172
Mean MCC LinearSVC: 0.27648358328670924
Mean AUC KNeighborsClassifier: 0.5600993888991381
Mean prAUC KNeighborsClassifier: 0.1251093078587185
Mean MCC KNeighborsClassifier: 0.056193542505766206
